In [1]:
import pandas as pd
import sqlite3
import numpy as np
from tqdm.notebook import tqdm
import os.path
from os import path


# 1 DocGraph_Hop_Teaming_2018

## 1.1 Read sample hop data

In [ ]:
#Sample filters
hop = pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv', 
                          nrows = 10000) #.query('transaction_count  >= 50').query('patient_count < 50')
hop= hop[(hop['transaction_count'] >= 25) & (hop['average_day_wait'] < 90 )]


In [ ]:
hop

## 1.2 Create hop_db database and load hop table

In [9]:
if path.exists("../data/hop_db.sqlite") == True:   
   with sqlite3.connect('../data/hop_db.sqlite') as db: 

    for chunk in tqdm(pd.read_csv('../data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)):
        chunk = chunk[(chunk['transaction_count'] >= 25 ) & chunk['average_day_wait'] < 90]       
        chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
        chunk.to_sql('hop', db, if_exists = 'append', index = False)  

0it [00:00, ?it/s]

# 2 Read NPPES file and load npi table  

## 2.1 Create a list for column names from npi file

In [5]:
result_cols =['NPI',
              'Entity Type Code',
              'Provider Organization Name (Legal Business Name)',
              'Provider Last Name (Legal Name)',
              'Provider First Name',
              'Provider Middle Name',
              'Provider Name Prefix Text',
              'Provider Name Suffix Text',
              'Provider Credential Text',
              'Provider First Line Business Practice Location Address',
              'Provider Second Line Business Practice Location Address',
              'Provider Business Practice Location Address City Name',
              'Provider Business Practice Location Address State Name',
              'Provider Business Practice Location Address Postal Code',
              'Healthcare Provider Taxonomy Code_1',
              'Healthcare Provider Taxonomy Code_2',
              'Healthcare Provider Taxonomy Code_3',
              'Healthcare Provider Taxonomy Code_4',
              'Healthcare Provider Taxonomy Code_5',
              'Healthcare Provider Taxonomy Code_6',
              'Healthcare Provider Taxonomy Code_7',
              'Healthcare Provider Taxonomy Code_8',
              'Healthcare Provider Taxonomy Code_9',
              'Healthcare Provider Taxonomy Code_10',
              'Healthcare Provider Taxonomy Code_11',
              'Healthcare Provider Taxonomy Code_12',
              'Healthcare Provider Taxonomy Code_13',
              'Healthcare Provider Taxonomy Code_14',
              'Healthcare Provider Taxonomy Code_15',
              'Healthcare Provider Primary Taxonomy Switch_1',
              'Healthcare Provider Primary Taxonomy Switch_2',
              'Healthcare Provider Primary Taxonomy Switch_3',
              'Healthcare Provider Primary Taxonomy Switch_4',
              'Healthcare Provider Primary Taxonomy Switch_5',
              'Healthcare Provider Primary Taxonomy Switch_6',
              'Healthcare Provider Primary Taxonomy Switch_7',
              'Healthcare Provider Primary Taxonomy Switch_8',
              'Healthcare Provider Primary Taxonomy Switch_9',
              'Healthcare Provider Primary Taxonomy Switch_10',
              'Healthcare Provider Primary Taxonomy Switch_11',
              'Healthcare Provider Primary Taxonomy Switch_12',
              'Healthcare Provider Primary Taxonomy Switch_13',
              'Healthcare Provider Primary Taxonomy Switch_14',
              'Healthcare Provider Primary Taxonomy Switch_15']

## 2.2 Read sample data

In [ ]:
# Read sample data
df = pd.read_csv('../NPPES_Data_Dissemination_February_2023/npidata_pfile_20050523-20230212.csv', 
                          nrows = 10000, usecols =['Provider Organization Name (Legal Business Name)'])


df

In [6]:
# Create a function to deal with the leading zeroes that are missing from zip codes

def zip_zeroes(x):
    if len(x) > 5:
        return x.zfill(9)[:5]
    elif len(x) > 0:
        return x.zfill(5)
    else: return np.nan

In [7]:

db = sqlite3.connect('../data/hop_db.sqlite')

In [8]:
# create the providers table from npi/nppes data and add it to the database
for chunk in tqdm(pd.read_csv('../NPPES_Data_Dissemination_February_2023/npidata_pfile_20050523-20230212.csv', 
                              usecols = result_cols,
                              chunksize = 100000,
                              low_memory = False)):
    
    # For providers that have indicated a primary taxonomy code, pull that code into a new column
    chunk['taxonomy_code'] = np.nan
    for n in range(1, 16):
        x = str(n)
        chunk.loc[chunk[f'Healthcare Provider Primary Taxonomy Switch_{x}'] == 'Y', 
                       'taxonomy_code'] = chunk[f'Healthcare Provider Taxonomy Code_{x}']
        
    # For providers that do not indicate a primary taxonomy code, pull the code from the first taxonomy column
    chunk.loc[chunk['taxonomy_code'].isna(), 'taxonomy_code'] = chunk['Healthcare Provider Taxonomy Code_1']
    
    # Drop the columns no longer needed
    chunk = chunk.drop(columns = ['Healthcare Provider Taxonomy Code_1', 'Healthcare Provider Primary Taxonomy Switch_1',
                                  'Healthcare Provider Taxonomy Code_2', 'Healthcare Provider Primary Taxonomy Switch_2',
                                  'Healthcare Provider Taxonomy Code_3', 'Healthcare Provider Primary Taxonomy Switch_3',
                                  'Healthcare Provider Taxonomy Code_4', 'Healthcare Provider Primary Taxonomy Switch_4',
                                  'Healthcare Provider Taxonomy Code_5', 'Healthcare Provider Primary Taxonomy Switch_5',
                                  'Healthcare Provider Taxonomy Code_6', 'Healthcare Provider Primary Taxonomy Switch_6',
                                  'Healthcare Provider Taxonomy Code_7', 'Healthcare Provider Primary Taxonomy Switch_7',
                                  'Healthcare Provider Taxonomy Code_8', 'Healthcare Provider Primary Taxonomy Switch_8',
                                  'Healthcare Provider Taxonomy Code_9', 'Healthcare Provider Primary Taxonomy Switch_9',
                                  'Healthcare Provider Taxonomy Code_10', 'Healthcare Provider Primary Taxonomy Switch_10',
                                  'Healthcare Provider Taxonomy Code_11', 'Healthcare Provider Primary Taxonomy Switch_11',
                                  'Healthcare Provider Taxonomy Code_12', 'Healthcare Provider Primary Taxonomy Switch_12',
                                  'Healthcare Provider Taxonomy Code_13', 'Healthcare Provider Primary Taxonomy Switch_13',
                                  'Healthcare Provider Taxonomy Code_14', 'Healthcare Provider Primary Taxonomy Switch_14',
                                  'Healthcare Provider Taxonomy Code_15', 'Healthcare Provider Primary Taxonomy Switch_15'])
     
    #clean up zip code column by putting missing leading zeroes back and getting the 9-digit entries down to 5
    chunk['Provider Business Practice Location Address Postal Code'] = chunk['Provider Business Practice Location Address Postal Code'].astype(str).apply(zip_zeroes)
    
    # Clean up the column names
    chunk.columns = [x.lower()
                     .replace('provider ', '')
                     .replace('business ', '')
                     .replace('practice ', '')
                     .replace(' text', '')
                     .replace(' (legal name)', '')
                     .replace(' ', '_') for x in chunk.columns]                  
   
    # append to provider table
    chunk.to_sql('npi', db, if_exists = 'append', index = False)   

0it [00:00, ?it/s]

In [10]:
db.close()

# 3. taxonomy file

## 3.1 Sample read

In [ ]:
tax_deets = pd.read_csv('../data/nucc_taxonomy_230.csv')


## 3.2 Load table taxonomy

In [11]:
#uncomment  
with sqlite3.connect('../data/hop_db.sqlite') as db: 

    for chunk in tqdm(pd.read_csv('../data/nucc_taxonomy_230.csv', chunksize = 10000)):             
        chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]      # Clean up the column names
        # make the taxonomy code column name match the taxonomy code column name in the provider table
        chunk = chunk.rename(columns = {'code' : 'taxonomy_code'})

        chunk.to_sql('taxonomy', db, if_exists = 'append', index = False)    

0it [00:00, ?it/s]

# 4 ZIP_CBSA file

## 4.1 Read sample ZIP_CBSA

In [ ]:
df = pd.read_excel('../data/ZIP_CBSA_122021.xlsx')

In [ ]:
df

## 4.2 Load zip_cbsa table

In [12]:
with sqlite3.connect('../data/hop_db.sqlite') as db: 
    df_zip = pd.read_excel('../data/ZIP_CBSA_122021.xlsx')
    df_zip = df_zip.rename(columns = {'usps_zip_pref_city' : 'city','usps_zip_pref_state' : 'state'})
    #get the leading zeroes back in place
    df_zip['zip'] = df_zip['zip'].astype(str).str.zfill(5)       
    df_zip.to_sql('zip_cbsa', db, if_exists = 'append', index = False) 